In [1]:
pip install streamlit imbalanced-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 492.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.0 MB/s eta 0:00:00


In [6]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score

st.title("Credit Card Fraud Detection")

# File uploader
uploaded_file = st.file_uploader("Upload creditcard.csv dataset", type=["csv"])

@st.cache_data
def load_data(file):
    df = pd.read_csv(file)
    df = df.dropna(subset=['Class'])
    return df

if uploaded_file:
    df = load_data(uploaded_file)
    st.write("### Dataset Preview")
    st.dataframe(df.head())

    # Show class distribution pie chart
    st.write("### Class Distribution")
    fig, ax = plt.subplots()
    df['Class'].value_counts().plot.pie(autopct='%1.1f%%', labels=['Non-Fraud', 'Fraud'], colors=['skyblue', 'orange'], ax=ax)
    ax.set_ylabel('')
    st.pyplot(fig)

    # Downsample majority class
    fraud = df[df['Class'] == 1]
    non_fraud = df[df['Class'] == 0].sample(n=1000, random_state=42)
    df_sampled = pd.concat([fraud, non_fraud])

    X = df_sampled.drop('Class', axis=1)
    y = df_sampled['Class']

    # Feature correlation heatmap
    st.write("### Feature Correlation Heatmap")
    fig2, ax2 = plt.subplots(figsize=(10, 7))
    sns.heatmap(df_sampled.corr(), cmap='coolwarm', center=0, ax=ax2)
    st.pyplot(fig2)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Define models
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100),
        'AdaBoost': AdaBoostClassifier(n_estimators=100),
        'KNN (K=1)': KNeighborsClassifier(n_neighbors=1),
        'Decision Tree': DecisionTreeClassifier(),
        'Extra Trees': ExtraTreesClassifier(n_estimators=100)
    }

    model_names = []
    accuracies = []
    roc_aucs = []
    best_model = None
    best_auc = 0
    best_probs = None
    best_preds = None

    st.write("## Model Training & Evaluation")

    for name, model in models.items():
        st.write(f"### Training {name}...")
        model.fit(X_scaled, y)
        y_pred = model.predict(X_scaled)
        y_prob = model.predict_proba(X_scaled)[:, 1]

        acc = accuracy_score(y, y_pred)
        auc = roc_auc_score(y, y_prob)

        st.write(f"Accuracy: {acc*100:.2f}%")
        st.write(f"ROC AUC: {auc:.4f}")

        st.text(classification_report(y, y_pred))

        model_names.append(name)
        accuracies.append(acc * 100)
        roc_aucs.append(auc)

        # Confusion matrix plot
        conf_matrix = confusion_matrix(y, y_pred)
        fig_cm, ax_cm = plt.subplots()
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", ax=ax_cm)
        ax_cm.set_title(f"Confusion Matrix - {name}")
        ax_cm.set_xlabel("Predicted")
        ax_cm.set_ylabel("Actual")
        st.pyplot(fig_cm)

        # ROC curve plot
        fpr, tpr, _ = roc_curve(y, y_prob)
        fig_roc, ax_roc = plt.subplots()
        ax_roc.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
        ax_roc.plot([0, 1], [0, 1], linestyle='--')
        ax_roc.set_title(f"ROC Curve - {name}")
        ax_roc.set_xlabel("False Positive Rate")
        ax_roc.set_ylabel("True Positive Rate")
        ax_roc.legend()
        st.pyplot(fig_roc)

        if auc > best_auc:
            best_auc = auc
            best_model = name
            best_probs = y_prob
            best_preds = y_pred

    # Accuracy Bar plot
    fig_acc, ax_acc = plt.subplots(figsize=(10, 5))
    sns.barplot(x=model_names, y=accuracies, palette="viridis", ax=ax_acc)
    ax_acc.set_ylabel("Accuracy (%)")
    ax_acc.set_title("Model Accuracy Comparison")
    ax_acc.set_ylim(90, 100)
    ax_acc.set_xticklabels(ax_acc.get_xticklabels(), rotation=45)
    st.pyplot(fig_acc)

    # ROC AUC Bar plot
    fig_auc, ax_auc = plt.subplots(figsize=(10, 5))
    sns.barplot(x=model_names, y=roc_aucs, palette="rocket", ax=ax_auc)
    ax_auc.set_ylabel("ROC AUC Score")
    ax_auc.set_title("Model ROC AUC Comparison")
    ax_auc.set_ylim(0.90, 1.0)
    ax_auc.set_xticklabels(ax_auc.get_xticklabels(), rotation=45)
    st.pyplot(fig_auc)

    # Line plot of best model probabilities
    st.write(f"### Sorted Fraud Probabilities - Best Model: {best_model}")
    fig_prob, ax_prob = plt.subplots(figsize=(10, 4))
    ax_prob.plot(np.sort(best_probs), label="Predicted Probabilities")
    ax_prob.set_xlabel("Sample Index")
    ax_prob.set_ylabel("Fraud Probability")
    ax_prob.legend()
    st.pyplot(fig_prob)

else:
    st.info("Please upload the creditcard.csv dataset to get started.")


2025-07-20 11:25:06.451 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.452 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.455 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.456 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-20 11:25:06.458 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar